# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
seed = 0
env = UnityEnvironment(file_name="Banana.app",seed = seed)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# # reset the environment
# env_info = env.reset(train_mode=True)[brain_name]

# # number of agents in the environment
# print('Number of agents:', len(env_info.agents))

# # number of actions
# action_size = brain.vector_action_space_size
# print('Number of actions:', action_size)

# # examine the state space 
# state = env_info.vector_observations[0]
# print('States look like:', state)
# state_size = len(state)
# print('States have length:', state_size)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
# env_info = env.reset(train_mode=False)[brain_name] # reset the environment
# state = env_info.vector_observations[0]            # get the current state
# score = 0                                          # initialize the score
# while True:
#     action = np.random.randint(action_size)        # select an action
#     env_info = env.step(action)[brain_name]        # send the action to the environment
#     next_state = env_info.vector_observations[0]   # get the next state
#     reward = env_info.rewards[0]                   # get the reward
#     done = env_info.local_done[0]                  # see if episode has finished
#     score += reward                                # update the score
#     state = next_state                             # roll over the state to next time step
#     if done:                                       # exit loop if episode finished
#         break
    
# print("Score: {}".format(score))

When finished, you can close the environment.

In [6]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

### initializing the agent

In [7]:
from dqn_agent import Agent
import torch

In [8]:
def banana_dqn(episodes_num=2000, epsilon_start=1.0, epsilon_min=0.01, epsilon_decay=0.995):
    scores = []
    score_window = deque(maxlen=100)
    epsilon = epsilon_start
    max_score = 15.0
    with open('testfile.txt','w') as reward_file:
        for episode in range(1, episodes_num+1):
            # reset the environment
            env_info = env.reset(train_mode=True)[brain_name] # reset the environment
            state = env_info.vector_observations[0]            # get the current state
            
            score = 0
            while True:
                #1 take action from DQN
                action = agent.act(state, eps= epsilon)              # select an action
                print(action)
                env_info = env.step(action)[brain_name]        # send the action to the environment
                next_state = env_info.vector_observations[0]   # get the next state
                reward = env_info.rewards[0]                   # get the reward
                done = env_info.local_done[0]                  # see if episode has finished
                agent.step(state, action, reward, next_state, done)
                score += reward                                # update the score
                reward_file.write("total_score:{}\treward:{}\taction:{}\n".format(score, reward, action))
                state = next_state                             # roll over the state to next time step
                if done:                                       # exit loop if episode finished
                    break
            score_window.append(score)
            scores.append(score)
            epsilon = max(epsilon_min, epsilon*epsilon_decay)
            
            print('\rEpisode: {}\tAverage Score: {:.2f}, score:{:.2f}'.format(episode, np.mean(score_window), score), end="")
            reward_file.write("Episode :{}\tAverage Score:{}\n".format(episode, np.mean(score_window)))
            if episode % 100 == 0:
                print('\rEpisode: {}\tAverage Score: {:.2f}'.format(episode, np.mean(score_window)))
            if np.mean(score_window)>= max_score:
                torch.save(agent.QN_local.state_dict(), 'checkpoint.pth')
                break
    return scores

env_info = env.reset(train_mode=True)[brain_name]
action_size = brain.vector_action_space_size
state_size = len(env_info.vector_observations[0])
print('Action Size:{}, State Size:{}'.format(action_size, state_size))
agent = Agent(state_size, action_size, seed = seed, training = True, pixels=False)
scores = banana_dqn()


Action Size:4, State Size:37
tensor([[ 0.0931,  0.0915, -0.1285,  0.0675]], grad_fn=<ThAddmmBackward>)
3
tensor([[ 0.0875,  0.0802, -0.1260,  0.0662]], grad_fn=<ThAddmmBackward>)
3
tensor([[ 0.0939,  0.0771, -0.1207,  0.0581]], grad_fn=<ThAddmmBackward>)
2
tensor([[ 0.0875,  0.0802, -0.1260,  0.0662]], grad_fn=<ThAddmmBackward>)
2
tensor([[ 0.0973,  0.0957, -0.1306,  0.0748]], grad_fn=<ThAddmmBackward>)
1
tensor([[ 0.1357,  0.1152, -0.1107,  0.0447]], grad_fn=<ThAddmmBackward>)
2
tensor([[ 0.1125,  0.0881, -0.1389,  0.0410]], grad_fn=<ThAddmmBackward>)
0
tensor([[ 0.0960,  0.0494, -0.1826,  0.0595]], grad_fn=<ThAddmmBackward>)
2
tensor([[ 0.1079,  0.0754, -0.1339,  0.0775]], grad_fn=<ThAddmmBackward>)
1
tensor([[ 0.1425,  0.1004, -0.1023,  0.0635]], grad_fn=<ThAddmmBackward>)
0
tensor([[ 0.0924,  0.0589, -0.1640,  0.0635]], grad_fn=<ThAddmmBackward>)
2
tensor([[ 0.0789,  0.0601, -0.1388,  0.0924]], grad_fn=<ThAddmmBackward>)
0
tensor([[ 0.0771,  0.0443, -0.2229,  0.0643]], grad_fn=<ThA

tensor([[ 0.1585,  0.1233, -0.1199,  0.0579]], grad_fn=<ThAddmmBackward>)
0
tensor([[ 0.0804,  0.0644, -0.1353,  0.0759]], grad_fn=<ThAddmmBackward>)
0
tensor([[ 0.0803,  0.0558, -0.1966,  0.0750]], grad_fn=<ThAddmmBackward>)
1
tensor([[ 0.1193,  0.0951, -0.1257,  0.0617]], grad_fn=<ThAddmmBackward>)
2
tensor([[ 0.1037,  0.0853, -0.1278,  0.0524]], grad_fn=<ThAddmmBackward>)
1
tensor([[ 0.1552,  0.1234, -0.1282,  0.0498]], grad_fn=<ThAddmmBackward>)
3
tensor([[ 0.1325,  0.0993, -0.1128,  0.0690]], grad_fn=<ThAddmmBackward>)
0
tensor([[ 0.0841,  0.0586, -0.1546,  0.0825]], grad_fn=<ThAddmmBackward>)
3
tensor([[ 0.0881,  0.0641, -0.1387,  0.0529]], grad_fn=<ThAddmmBackward>)
0
tensor([[ 0.0843,  0.0569, -0.1952,  0.0708]], grad_fn=<ThAddmmBackward>)
1
tensor([[ 0.1161,  0.0960, -0.1254,  0.0603]], grad_fn=<ThAddmmBackward>)
2
tensor([[ 0.1035,  0.0857, -0.1255,  0.0534]], grad_fn=<ThAddmmBackward>)
1
tensor([[ 0.1528,  0.1253, -0.1254,  0.0488]], grad_fn=<ThAddmmBackward>)
0
tensor([[ 0.

KeyboardInterrupt: 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)),scores)
plt.xlabel('Episode ')
plt.ylabel('Score')
plt.show()


## Test

In [ ]:

for i in range(5):
    env_info = env.reset(train_mode=False)[brain_name] # reset the environment
    action_size = brain.vector_action_space_size
    state_size = len(env_info.vector_observations[0])

    agent = Agent(state_size, action_size, seed = seed, training = False)
    agent.QN_local.load_state_dict(torch.load('checkpoint.pth'))

    state = env_info.vector_observations[0]            # get the current state
    score = 0                                          # initialize the score
    while True:
        action = agent.act(state, eps=.0)                      # select an action
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break

    print("Test: {} Score: {}".format(i, score))

In [ ]:
env.close()